# Install & Import

The following instructions assume you have completed the setup (Steps I - IV and VI [here](https://github.com/Datican-NG/hpc-docs/blob/main/slurm.md)).
1. In a terminal window on `randi`, execute the following commands to create and activate the Slideflow Conda environment:

   ```bash
   conda create -y --name sflow python=3.9
   conda activate sflow
   pip install slideflow[torch] cucim cupy-cuda11x ipykernel 
   ```

   Note: You only need to create the environment once. For future use, simply activate it with `conda activate slideflow`.

2. Check if you are already on a compute node. The green or blue box in the lower left hand corner in VSCode shows your `HOSTNAME` (i.e., the name of the machine you are on). It should be `cri22cnXXX` or `cri22inXXX`. If it has a "cn" in it, it is a compute node; if it has an "in" in it, it is the head node. You need to run this notebook on a compute node because it requires a GPU.

3. If you are not on a compute node, follow these instructions to start a VSCode session on a compute node:
   
   a. In Command Prompt (Windows) or Terminal (Mac/Linux) on your local machine, SSH into the cluster login node: `ssh randi`

   b. Request a compute node:
     ```bash
     srun -p gpuq --gres=gpu:1 -t 640:00 --cpus-per-task 4 --pty /bin/bash
     ```
     Your prompt will change to `USERNAME@hostname` upon success.

     c. In VSCode, click the remote-ssh box (green or blue box in the lower left corner) and select "Connect to Host". Enter `HOSTNAME`, replacing `HOSTNAME` with the value from step 2.
     
     d. Navigate back to the notebook and open it.

4. In the top right corner of this notebook, select "Select Kernel -> slideflow".

In [ ]:
# Import slideflow and confirm the version.
#from PIL import Image
import slideflow as sf
import os
sf.about()

# Create a Project

First, we'll start by creating a project. We'll need a path to where the whole-slide images (WSI) are stored, and a clinical annotations file that contains our patient-level outcomes, in CSV format. See [Setting up a Project](https://slideflow.dev/project_setup/) for more information.

In [ ]:
# Create the project.
sf.create_project(
    root=f'/gpfs/data/datican-lab/workspaces/{os.environ["USER"]}/slideflow_v1',
    slides='/gpfs/data/datican-lab/slides/images/',
    annotations='/gpfs/data/datican-lab/slides/anns/HNSC annotations.csv'
)

In [ ]:
# Load the created project.
project = sf.load_project(f'/gpfs/data/datican-lab/workspaces/{os.environ["USER"]}/slideflow_v1')

# Prepare a Dataset

Next, we'll want to prepare a Dataset, a collection of whole-slide images and associated tiles extracted from these WSIs. For this experiment, we'll prepare a dataset of 299 x 299 pixel image tiles, with an optical magnification of 302 x 302 microns (1 micron-per-pixel, equivalent to 10X magnification).

See [Datasets](https://slideflow.dev/datasets_and_val/) for more information on dataset handling.

In [ ]:
# Prepare a dataset of slides, defining
# the expected image tile size in magnification (micron, tile_um)
# and resolution (pixels, tile_px)
dataset = project.dataset(tile_px=299, tile_um=302)

Let's confirm everything is working so far by retrieving the first slide in our dataset and looking at the thumbnail.

In [ ]:
# Get a list of all downloaded slides, and show a thumbnail
# of the first slide.
slide_path = dataset.slide_paths()[0]
wsi = sf.WSI(slide_path, tile_px=299, tile_um=302)
wsi.thumb()

We'll want to section this large whole-slide image into smaller tiles, for downstream model training. Let's preview what tile extraction would look like at this magnification level.

In [ ]:
# Show a preview of tile extraction for this whole-slide image.
wsi.preview()

Note that because we do not have pathologist-annotated Regions of Interest (ROIs), we are extracting tiles from the whole slide (excluding background). 

## Extract tiles

Next, we'll want to extract tiles from our dataset, storing them in TFRecord format. We'll use the Otsu thresholding algorithm to remove background, and we'll save images in JPEG format for space efficiency. See [Slide Processing](https://slideflow.dev/slide_processing/) for more information on slide processing options and tile extraction.

This step can take quite a bit of time, depending on your dataset size, storage speed, image format, and compute. As a rough guideline, expect that it will take 3-5 seconds per slide.

In [ ]:
# Extract image tiles from whole-slide images,
# exporting image tiles into buffered TFRecord format.
dataset.extract_tiles(qc='otsu', grayspace_fraction=1, img_format='jpg')

After tiles have been extracted, be sure to check the [tile extraction PDF report](https://slideflow.dev/slide_processing/#extraction-reports), which can provide important insights into how this process went and if there are any issues. This step can help you identify whether different slide processing / filtering may be necessary.

For this project, the PDF report will be in `project/tfrecords/299px_302um/tile_extraction_report*.pdf`. There will also be a spreadsheet (CSV) generated that logs the tile extraction settings used for each slide, located here at `project/tfrecords/299px_302um/extraction_report.csv`.

## Identifying and fixing issues

As an example, the PDF report has identified a potential issue with the slide `TCGA-F7-8298-01Z-00-DX1.e9df5104-c9a0-43af-a2e8-86d3f4029e03`. Let's load up this specific slide, apply the slide processing we used during tile extraction, and see if we can identify the problem.

In [ ]:
# Find the path to the slide in question and load it.
slide_path = dataset.find_slide(slide='TCGA-F7-8298-01Z-00-DX1.e9df5104-c9a0-43af-a2e8-86d3f4029e03')
wsi = sf.WSI(slide_path, tile_px=299, tile_um=302)

# Apply Otsu Thresholding.
wsi.qc('otsu')

# Preview tile extraction.
wsi.preview(grayspace_fraction=1)

Yes, there is a problem - there are no tiles extracted within the tissue, only within the pen marks. Why? Probably because the Otsu's thresholding algorithm got confused by the huge, bold pen marks and thought *that* was the foreground. We can confirm this by looking at the slide QC mask:

In [ ]:
Image.fromarray(wsi.qc_mask)

Indeed, the issue is that the algorithm identified the pen marks as foreground (black). So we need to try a different QC algorithm.

One of the solutions we have identified to this problem is by performing Gaussian (blur) filtering, which can remove both blurry regions (like pen marks) as well as background. Let's see if that works in this case.

In [ ]:
# Import the slideflow QC submodule.
from slideflow.slide import qc

# Remove the prior QC
wsi.remove_qc()

# Apply Gaussian blur filtering to the slide
wsi.qc(qc.GaussianV2())  # V2 is faster than the original implementation

This looks promising; let's see what the tile extraction would look like.

In [ ]:
wsi.preview(grayspace_fraction=1)

It's better, but not perfect. Some additional exploration might need to be done into other algorithms (or a combination of algorithms). Or, if a suitable fully automated algorithm cannot be found, a Region of Interest (ROI) can be drawn around the areas of tissue, excluding the pen marks. You can also leverage a DL segmentation model (like U-Net) to automatically draw ROIs. See [Tissue Segmentation](https://slideflow.dev/segmentation/) for more information.

See [Masking & Filtering](https://slideflow.dev/slide_processing/#masking-filtering) for a more in-depth discussion of various automated slide filtering options, and [Regions of Interest](https://slideflow.dev/slide_processing/#regions-of-interest) for an overview of how ROIs can be generated and applied.

Once the desired slide filtering / QC pipeline is idenfied, tiles can be re-extracted using this new strategy. For the purposes of this example, we'll continue onward using the flawed data.

## Inpsecting the dataset

Let's take a look at the dataset summary to get an idea of how many patients, slides, and tiles we have in our dataset.

In [ ]:
# Print a summary of the dataset,
# showing how many slides are included
# and how many tiles were extracted from slides.
dataset.summary()

So we have 869,047 tiles across 459 patients. This will be more than sufficient for training models.

# Train a tile-based model

Next, for the purposes of this example, we'll train a simple tile-based model with the binary categorization task of predicting the presence of Human Papillomavirus (HPV). Each patient's HPV status is recoreded in our project annotations file, under the column `"HPV_status"`.

First, we'll split our dataset into training (70%) and testing (30%). The training data will be further split into training and validation. See our [documentation](https://slideflow.dev/datasets_and_val/#training-validation-splitting) for more details and examples regarding dataset splitting.

Of note - for this example, we will be training with simple dataset splits, balanced by the outcome of interest (HPV status). For simplicity, we are not using site-preserved splits, which are important when training on multi-institution data like TCGA to avoid being led astray by [confounding, site-related batch effects](https://www.nature.com/articles/s41467-021-24698-1). Site-preserved splitting requires installation of either [CPLEX or Pyomo/Bonmin](https://slideflow.dev/installation/#optional), and can be enabled with the argument `strategy='k-fold-preserved-site'`.

In [ ]:
# Train models.

# First, split the data into training (70%) / testing (30%).
# Ensure equal balancing according to the outcome 'subtype'.
train, test = dataset.split(val_fraction=0.3, labels='HPV_status')

# Next, we'll reserve 10% of the training data for validation.
train, val = train.split(val_fraction=0.1, labels='HPV_status')

Now, we'll need to configure our model hyperparameters and begin training. We'll train a classic ResNet50 model. There are many tunable hyperparameters; see [Training](https://slideflow.dev/training/) for more information. Because each slide has an abundance of redundant morphologic information spread across the image tiles, we don't typically need to train for more than about 1 epoch.

In [ ]:
# Train a tile-based model.

# Configure the hyperparameters.
hp = sf.ModelParams(
    tile_px=299,      # Tile width, in pixels.
    tile_um=302,      # Tile width, in microns (effective magnification).
    augment='xyrjb',  # Augment with random flip/rotation, JPEG compression, and Guassian blur.
    batch_size=128,   # Batch size of 128. Adjust/tune as needed.
    model='resnet50', # Use the ResNet50 architecture.
    epochs=1,         # Train for 1 epoch.
    dropout=0.2,      # Dropout, p=0.2.
)

# Start training.
project.train(
    outcomes='HPV_status', # Ground-truth labels.
    params=hp,             # Hyperparameters.
    dataset=train,         # Training dataset.
    val_dataset=val,       # Validation dataset.
    validate_on_batch=256, # Run validation every 256 training steps.
    validation_steps=32    # When validating, run 32 batches at a time.
  )

## Held-out testing

At this point, we would continue experimenting with hyperparameter settings, architecture, slide processing methods, and magnification levels to get a better understanding of what the best settings are for this outcome of interest. Once all of these investigations are complete, we'll want to test the final model on our held-out test set. 

Trained models will be saved in the `models` folder within the main project directory. See [Evaluation](https://slideflow.dev/evaluation/) for more information on evaluating models.

In [ ]:
# Now, let's generate predictions
saved_model = 'project/models/00000-HPV_status-HP0/HPV_status-HP0_epoch1.zip'
project.evaluate(saved_model, 'HPV_status', dataset=test)

Despite being trained on known flawed data with potential confounders (pen marks), our model has done well on the held out test set, with an Area Under Receiver Operator Curve (AUROC) of 0.83.

With careful hyperparameter tuning, dataset preparation, and [uncertainty quantification](https://slideflow.dev/uq/), we have shown that it is possible to build models on this outcome reaching an AUROC of ~0.87 (see: [paper](https://bmcbioinformatics.biomedcentral.com/articles/10.1186/s12859-024-05758-x)). 

# Multiple-instance learning

An alternative to tile-based models is [Multiple-Instance Learning (MIL)](https://slideflow.dev/mil/). With MIL, each tile is first converted into a feature vector, and feature vectors from all tiles in a slide are grouped together in a "bag". The MIL architecture reduces all the feature vectors from a slide into a single vector using attention weighting, and then generates a prediction from the reduced vector. The key difference with this approach, compared to a tile-based model, is that the MIL model "sees" all areas of a slide when making a prediction. In contrast, tile-based models make predictions on each image tile in isolation, and the final predictions are merged through simple average.

The first step in training an MIL model is to convert tile images into feature vectors. There are many publicly available pathology-specific pretrained feature extractors. Slideflow supports four out of the box: [CTransPath](https://github.com/Xiyue-Wang/TransPath), [RetCCL](https://github.com/Xiyue-Wang/RetCCL), [HistoSSL](https://github.com/owkin/HistoSSLscaling), and [PLIP](https://github.com/PathologyFoundation/plip). However, you can also easily implement your own [custom feature extractor](https://slideflow.dev/custom_extractors/), if you have the weights to a pretrained model. You can also train your own using [Self-Supervised Learning](https://slideflow.dev/ssl/) (either SimCLRv2 or DINOv2).

For this example, we will use the popular CTransPath model.

In [ ]:
# Train an MIL model.
import slideflow.mil

In [ ]:
# Before training an MIL model, we need to convert images into feature vectors.
# We'll use the pretrained CTransPath [1] pathology feature extractor.
# 1) https://doi.org/10.1016/j.media.2022.102559
ctranspath = sf.model.build_feature_extractor('ctranspath', tile_px=299)

Now that we've built our feature extractor, we'll use it to convert all the images in our dataset into vectors. These will be saved in the `pt_files` folder within the main project directory.

In [ ]:
# Generate and save CTransPath features
# for all images in the dataset.
bags = project.generate_feature_bags(ctranspath, dataset=dataset)

For this example, we'll train a simple attention-based MIL model. See [Multiple-Instance Learning](https://slideflow.dev/mil/) for a discussion of other available architectures and hyperparameters.

In [ ]:
# Prepare hyperparameters for MIL training.
config = sf.mil.mil_config('attention_mil', lr=1e-4, fit_one_cycle=False, epochs=10)

In [ ]:
# Start MIL model training.
project.train_mil(config, train, val, outcomes='HPV_status', bags=bags)

In [ ]:
# Test performance on the held-out test set.
saved_model = 'project/mil/00000-attention_mil-HPV_status'
project.evaluate_mil(saved_model, 'HPV_status', dataset=test, bags=bags)

As with the tile-based model, we're seeing overall good results, with an AUROC of 0.837 on the held-out test set. With careful hyperparameter tuning, feature extractor selection, and decisions regarding tile magnification and stain normalization, this can likely be improved.

# Post-Training Interpretability

Although not well suited to a Jupyter notebook, [Slideflow Studio](https://slideflow.dev/studio/) provides an interactive, graphical user interface for generating model predictions across whole-slide images. Prior to training, Studio can be used for dataset curation and cleaning, allowing you to draw/edit ROIs, visualize the effect of varying slide processing methods, and generate ROIs through tissue segmentation models. After training, you can use Studio to interact with heatmaps or model predictions and/or attention. 

We highly recommend sitting down with a pathologist to review prediction/attention heatmaps, which is an excellent way to determine whether the model is making biologically-plausible predictions. It can also highlight previously undiscovered batch effects or confounders.

For our Studio demo, let's find an example HPV positive and HPV negative case. Each dataset contains a pandas dataframe with clinical outcomes; we'll use this to find our example patients.

In [ ]:
# Get the pandas dataframe of clinical annotations
# in this test dataset.
df = test.filtered_annotations

# Show the HPV positive patients.
df.loc[df.HPV_status == 'positive']

In [ ]:
# Show the HPV negative patients.
df.loc[df.HPV_status == 'negative']

# High-Dimensional Feature Exploration

There are many additional analytical tools in Slideflow beyond just dataset preparation and model training. For example, Slideflow includes tools for interacting with and visualizing high-dimensional feature vectors (e.g. those generated for MIL training). Generating UMAP plots of feature vectors provides a flexible tool for understanding the distribution of your data, probing for signal, and investigating for batch effects.

## Generating & Visualizing UMAPs

In [ ]:
# Next, we'll do some basic feature exploration by reducing the dimensionality
# of the feature space with UMAP and visualize with plotting.
features = sf.DatasetFeatures.from_bags(bags)
slide_map = features.map_activations()

Our UMAP-reduced feature space is now saved in the `slide_map` object. Let's take a look at our CTransPath vector distribution.

In [ ]:
# Label and plot the UMAP.
labels, unique = dataset.labels('HPV_status', format='name')
slide_map.label_by_slide(labels)
slide_map.plot(subsample=5000)

It looks like there are two primary clusters in this dataset, and they do not seem to be associated with our outcome of interest (HPV). One of the first things we investigate when assessing clusters is to look for potential batch effects. Let's see if the site of origin (the hospital the tissue came from) is a confounder in this dataset.

In [ ]:
# Label and plot the UMAP.
labels, unique = dataset.labels('site', format='name')
slide_map.label_by_slide(labels)
slide_map.plot(subsample=5000)

Indeed, site of origin is a major confounder, and appears to be the primary association behind the clustering we see.

## Mosaic Maps

For added interpretability, Slideflow can convert a UMAP into a "mosaic map" by overlaying representative image tiles. This can help provide a better understanding of the role of color, confounders, and obvious tissue morphology in the observed distribution/clustering of the data.

In [ ]:
# Plot the dimensionality-reduced UMAP of CTransPath features
# as a mosaic, by overlaying representative image tiles.
mosaic = slide_map.build_mosaic(tfrecords=dataset.tfrecords())
mosaic.plot(figsize=(15, 15))

Here, we can see that some of the clusters (e.g. the far bottom left) are clusters of artifactual pen marks. The primary and secondary clusters appear to be strongly associated with visually apparent color differences, consistent with the observation that these are associated with site of origin.

## Using these insights

From these UMAP investigations, we've found evidence that there is site-related batch effect, color/stain-related batch effects, and pen mark batch effects. Knowing this, we can revisit our original dataset preparation and try the following:

1. Experiment with different stain normalization strategies, to see if we can reduce the color effects in our visualized UMAPs
2. Train models with site-preserved dataset splits, to avoid site-related confounders
3. Improve our dataset QC with better pen mark removal

Once we've done these things, we can try generating new UMAPs and see if things improve. Once things have improved, we can re-try model training. 

# Summary

As we've seen, building stable and reliable models from digital pathology images is an iterative process, requiring tuning of dataset preparation methods, stain/color normalization, understanding of batch effects, hyperparameter tuning, and robust evaluation. Slideflow provides a large array of tools to support researchers through each of these steps. Although there is no "one size fits all" solution for building reliable pathology models, I hope this tutorial has been helpful in providing you with a framework for how to think through the necessary experimental process and has provided a foundation for future learning and development.

For more information, please see the following:
- Docs: https://slideflow.dev
- GitHub: https://github.com/jamesdolezal/slideflow
- Paper: https://bmcbioinformatics.biomedcentral.com/articles/10.1186/s12859-024-05758-x

And should you have any questions, please do not hesitate to reach out to me directly at james@slideflow.dev.

All the best,

James